In [31]:
import os
# insert token
hf_token = os.getenv("HF_TOKEN")
assert hf_token, "HF_TOKEN is not set"

# Authenticate to access Google Cloud resources
from google.colab import auth
auth.authenticate_user()


In [14]:
!apt-get install -y python3-dev
!pip install --no-binary :all: crcmod

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for crcmod: filename=crcmod-1.7-cp311-cp311-linux_x86_64.whl size=31658 sha256=43a34b56d31267cf01452ffcb15533388c3095d49d976f7d64c06d3b2ebf9932
  Stored in directory: /root/.cache/pip/wheels/23/94/7a/8cb7d14597e6395ce969933f01aed9ea8fa5f5b4d4c8a61e99
Successfully built crcmod


In [15]:

from google.colab import auth
auth.authenticate_user()

from google.cloud import storage
from pathlib import Path
import os

In [10]:
import os
from pathlib import Path
import torch
import timm
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import subprocess

# Set paths
gcs_bucket = "bracs-dataset-bucket"
tile_root_gcs = f"gs://{gcs_bucket}/Tiles/train"
embedding_root_gcs = f"gs://{gcs_bucket}/Embeddings/train"
tile_root_local = Path("/content/tiles/train")
embedding_root_local = Path("/content/embeddings/train")

tile_root_local.mkdir(parents=True, exist_ok=True)
embedding_root_local.mkdir(parents=True, exist_ok=True)

# Load model
tile_encoder = timm.create_model("hf_hub:prov-gigapath/prov-gigapath", pretrained=True).cuda().eval()
print("✅ Tile Encoder loaded.")
print("🧮 Total parameters:", sum(p.numel() for p in tile_encoder.parameters()))

# Image transform
transform = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])


✅ Tile Encoder loaded.
🧮 Total parameters: 1134953984


In [28]:
def encode_slide_tiles(slide_id: str):
    slide_tile_dir = tile_root_local / slide_id
    save_path = embedding_root_local / slide_id / f"{slide_id}_embeddings.pt"
    save_path.parent.mkdir(parents=True, exist_ok=True)

    image_paths = sorted(slide_tile_dir.glob("*.png"))
    print(f"📂 {slide_id}: {len(image_paths)} tiles")

    all_embeddings = []
    all_coords = []

    for img_path in tqdm(image_paths, desc=f"🧠 Encoding {slide_id}"):
        img = Image.open(img_path).convert("RGB")
        x = transform(img).unsqueeze(0).cuda()

        with torch.no_grad():
            embed = tile_encoder(x).squeeze().cpu()

        # Extract (x, y) from filename: "x1234_y5678.png"
        name_parts = img_path.stem.split("_")
        x_coord = int(name_parts[0][1:])
        y_coord = int(name_parts[1][1:])

        all_embeddings.append(embed)
        all_coords.append([x_coord, y_coord])

        print(f"📏 {img_path.name}: embedding shape {embed.shape}")

    # Stack and save
    embeddings_tensor = torch.stack(all_embeddings)
    coords_tensor = torch.tensor(all_coords)

    print(f"📊 {slide_id}: embeddings shape {embeddings_tensor.shape}, coords shape {coords_tensor.shape}")

    torch.save({
        "embeddings": embeddings_tensor,
        "coords": coords_tensor
    }, save_path)

    print(f"✅ Saved slide embeddings to {save_path}")


In [23]:
def run_encoder_pipeline_from_gcs():
    # List all subfolders in the GCS bucket
    list_command = f"gsutil ls {tile_root_gcs}/"
    result = subprocess.run(list_command.split(), capture_output=True, text=True)
    slide_dirs = [line.strip().rstrip("/") for line in result.stdout.strip().split("\n")]

    for slide_path in slide_dirs:
        slide_id = Path(slide_path).name
        print(f"\n🔽 Downloading tiles for {slide_id}...")

        # Download to /content/tiles/train/BRACS_xxx
        local_slide_dir = tile_root_local / slide_id
        subprocess.run([
    "gsutil", "-m", "cp", "-r", str(slide_path), str(local_slide_dir.parent)
], check=True)

        # Run inference
        encode_slide_tiles(slide_id)

        # Upload embeddings
        gcs_target = f"{embedding_root_gcs}/{slide_id}"
        subprocess.run(["gsutil", "-m", "cp", "-r", str(embedding_root_local / slide_id), gcs_target], check=True)
        print(f"☁️ Uploaded embeddings for {slide_id} to {gcs_target}")

        # Cleanup
        subprocess.run(["rm", "-rf", str(local_slide_dir)])
        subprocess.run(["rm", "-rf", str(embedding_root_local / slide_id)])


In [29]:
run_encoder_pipeline_from_gcs()



🔽 Downloading tiles for BRACS_1003728...
📂 BRACS_1003728: 109 tiles


🧠 Encoding BRACS_1003728:   2%|▏         | 2/109 [00:00<00:06, 16.78it/s]

📏 x10304_y46144.png: embedding shape torch.Size([1536])
📏 x10304_y46592.png: embedding shape torch.Size([1536])
📏 x107968_y61824.png: embedding shape torch.Size([1536])


🧠 Encoding BRACS_1003728:   4%|▎         | 4/109 [00:00<00:05, 18.28it/s]

📏 x114240_y57792.png: embedding shape torch.Size([1536])


...

🧠 Encoding BRACS_1003728:  97%|█████████▋| 106/109 [00:05<00:00, 20.88it/s]

📏 x90048_y52416.png: embedding shape torch.Size([1536])
📏 x94080_y30016.png: embedding shape torch.Size([1536])
📏 x9856_y33152.png: embedding shape torch.Size([1536])
📏 x9856_y46144.png: embedding shape torch.Size([1536])


🧠 Encoding BRACS_1003728: 100%|██████████| 109/109 [00:05<00:00, 20.78it/s]

📏 x9856_y46592.png: embedding shape torch.Size([1536])
📊 BRACS_1003728: embeddings shape torch.Size([109, 1536]), coords shape torch.Size([109, 2])
✅ Saved slide embeddings to /content/embeddings/train/BRACS_1003728/BRACS_1003728_embeddings.pt


☁️ Uploaded embeddings for BRACS_1003728 to gs://bracs-dataset-bucket/Embeddings/train/BRACS_1003728
